In [9]:
import meta_config as mc
import abc
import numpy as np

In [10]:
# Create a fake third party library you might have imported
from types import SimpleNamespace
astrometryLibrary = SimpleNamespace()

class LibraryAstrometryConfig(mc.Config):
    '''This is like a third party library'''
    minNumAstrometryStars = mc.Field(doc="Minimum number of starts in astrometry", dtype=int, default=14)

def AstrometryLibraryFunction(config, image):
    '''This is like a third party library for astrometry'''
    pass

astrometryLibrary.LibraryAstrometryConfig = LibraryAstrometryConfig
astrometryLibrary.AstrometryLibraryFunction = AstrometryLibraryFunction

In [26]:
# Create a base photometry class
class Photometry(abc.ABC):
    def __init__(self, config):
        self.config = config
    
    @abc.abstractmethod
    def getFlux(self, image, config):
        pass

# Create two photometry classes, and configs for them
class PSFConfig(mc.Config):
    '''Configuration for PSF photometry'''
    renormalize = mc.Field(doc="Renormalize input PSF", dtype=bool, default=False)
    
class PSFRoutine(Photometry):
    ConfigClass = PSFConfig
    
    def getFlux(self, image, config):
        pass

class ApertureConfig(mc.Config):
    '''Configuration for Aperture Photometry'''
    doSkySubtraction = mc.Field(doc="Do local background subtraction", dtype=bool, default=True)

class ApertureRoutine(Photometry):
    ConfigClass = ApertureConfig
    
    def getFlux(self, image, config):
        pass

In [17]:
def LongExecutionFramework(config, extraArgs):
    astrometryLibrary.AstrometryLibraryFunction(config.astrometryConfig, extraArgs['image'])
    if config.doPhotometry:
        pass
    pass

In [27]:
import io
# Demo configuring with a file, not and not just in code
# Note, not all config options need be specified
configFile = io.StringIO("config.sigmaThreshold = 28")

In [29]:
class WorkFlowConfig(mc.Config):
    doPhotometry = mc.Field(doc="Turn photometry on or off", dtype=bool, default=True)
    imType = mc.ChoiceField(doc="Type of image", allowed={"raw": "uncalibrated image",
                                                          "calibrated": "calibrated image"},
                            default="raw", dtype=str)
    photometryMethod = mc.ConfigurableField(target=PSFRoutine, doc="The photometry that will be executed")
    sigmaThreshold = mc.RangeField(doc="how many background sigmas to threshold detections",
                                   min=0, max=100, default=5, dtype=float)
    astrometryConfig = mc.ConfigField(doc="Configuration for astrometry",
                                      dtype=astrometryLibrary.LibraryAstrometryConfig,
                                      default=astrometryLibrary.LibraryAstrometryConfig)

config = WorkFlowConfig()
# Demo
print(f"The default for psf renorm is {config.photometryMethod.renormalize}")
config.photometryMethod.retarget(ApertureRoutine)
print(f"The target Class is {config.photometryMethod.target}")

config.photometryMethod.doSkySubtraction = False
config.astrometryConfig.minNumAstrometryStars = 25

# Load in configs from a file
print(f"sigma is {config.sigmaThreshold}")
config.loadFromStream(configFile.getvalue())
print(f"new sigma is {config.sigmaThreshold}")

LongExecutionFramework(config, extraArgs={"image":np.arange(12*12).reshape(12,12)})

The default for psf renorm is False
The target Class is <class '__main__.ApertureRoutine'>
sigma is 5.0
new sigma is 28.0


In [14]:
config.history

{'astrometryConfig': [('config value set',
   [StackFrame(/usr/lib/python3.6/runpy.py, 193, _run_module_as_main),
    StackFrame(/usr/lib/python3.6/runpy.py, 85, _run_code),
    StackFrame(/usr/lib/python3/dist-packages/ipykernel_launcher.py, 16, <module>),
    StackFrame(/usr/lib/python3/dist-packages/traitlets/config/application.py, 658, launch_instance),
    StackFrame(/usr/lib/python3/dist-packages/ipykernel/kernelapp.py, 486, start),
    StackFrame(/usr/lib/python3/dist-packages/zmq/eventloop/ioloop.py, 177, start),
    StackFrame(/usr/lib/python3/dist-packages/tornado/ioloop.py, 888, start),
    StackFrame(/usr/lib/python3/dist-packages/tornado/stack_context.py, 277, null_wrapper),
    StackFrame(/usr/lib/python3/dist-packages/zmq/eventloop/zmqstream.py, 440, _handle_events),
    StackFrame(/usr/lib/python3/dist-packages/zmq/eventloop/zmqstream.py, 472, _handle_recv),
    StackFrame(/usr/lib/python3/dist-packages/zmq/eventloop/zmqstream.py, 414, _run_callback),
    StackFrame(/us

In [15]:
config.toDict()

{'astrometryConfig': {'minNumAstrometryStars': 25},
 'doPhotometry': True,
 'imType': 'raw',
 'photometryMethod': {'doSkySubtraction': False},
 'sigmaThreshold': 5.0}

In [16]:
import io

ioString = io.StringIO()

# Save a config out to a file
config.saveToStream(ioString)
print(ioString.getvalue())

import __main__
assert type(config)==__main__.WorkFlowConfig, 'config is of type %s.%s instead of __main__.WorkFlowConfig' % (type(config).__module__, type(config).__name__)
# Turn photometry on or off
config.doPhotometry=True

# Type of image
config.imType='raw'

config.photometryMethod.retarget(target=__main__.ApertureRoutine, ConfigClass=__main__.ApertureConfig)

# Do local background subtraction
config.photometryMethod.doSkySubtraction=False

# how many background sigmas to threshold detections
config.sigmaThreshold=5.0

# Minimum number of starts in astrometry
config.astrometryConfig.minNumAstrometryStars=25


